In [18]:
import pandas as pd
import numpy as np
import yaml
from scipy.optimize import minimize
from sklearn.metrics import mean_squared_error
from C_functions_opti import get_LHS_samples, model_optimization, plot_estimation

##### 1. Load all data that is needed

In [19]:
# Load experimental data
df_exp = pd.read_csv('data/data_combined.csv')
biomass_exp = df_exp['Biomass [g/L]']
substrate_exp = df_exp['Glucose [g/L]']

In [20]:
# Load parameters from YAML file
with open('config/parameters.yml', 'r') as file:
    param = yaml.safe_load(file)

mu_max = param['mu_max']
X_max = param['X_max']
Ks = param['Ks']
Ks_qs = param['Ks_qs']
Ki = param['Ki']
Yxs = param['Yxs']
qs_max = param['qs_max']
m_s = param['m_s']
lag = param['lag']

##### 2. Define all possible equations for mu and qs

In [21]:
# -- MONOD / insert: mu_max, c_glucose, Ks
mu0 = lambda mu_max, c_glucose, Ks: mu_max * c_glucose / (c_glucose + Ks)
# -- LOGISTIC / insert: mu_max, c_biomass, X_max
mu1 = lambda mu_max, c_biomass, X_max: mu_max * (1 - (c_biomass/ X_max)) 
# -- MONOD + LOGISTIC / insert: mu_max, c_glucose, Ks, c_biomass, X_max
mu2 = lambda mu_max, c_glucose, Ks, c_biomass, X_max: mu_max * (c_glucose / (c_glucose + Ks)) * (1 - (c_biomass/ X_max))
# -- MONOD + LOGISTIC + INHIBITION / insert: mu_max, c_glucose, Ks, Ki, c_biomass, X_max
mu3 = lambda mu_max, c_glucose, Ks, Ki, c_biomass, X_max: mu_max * (c_glucose / (c_glucose + Ks + (c_glucose**2/ Ki))) * (1 - (c_biomass/ X_max))

In [22]:
# -- MONOD / insert: qs_max, c_glucose, Ks_qs
qs0 = lambda qs_max, c_glucose, Ks_qs: qs_max * c_glucose / (Ks_qs + c_glucose)
# -- MONOD + NON COMPETITIVE INHIBITION / insert: qs_max, c_glucose, Ks_qs, Ki, glu_met
qs1 = lambda qs_max, c_glucose, Ks_qs, Ki, glu_met: qs_max * c_glucose / (Ks_qs + c_glucose) * (Ki / (Ki + glu_met))
# -- YIELD / insert: Yxs, f_glucose, V
qs2 = lambda Yxs, f_glucose, V: 1/Yxs * f_glucose / V #NOT SURE IF CORRECT because g_S/g_X but not per h
# -- MONOD + METABOLIZED GLU / insert: qs_max, c_glucose, Ks_qs, glu_met, lag
qs3 = lambda qs_max, c_glucose, Ks_qs, glu_met, lag: qs_max * c_glucose / (Ks_qs + c_glucose) * (1 / (np.exp(glu_met * lag)))

In [23]:
mu_all=[mu0, mu1, mu2, mu3]
qs_all=[qs0, qs1, qs2, qs3]

##### 3. Define functions that are needed

In [24]:
# Root mean squared error is the objective function
def objective_function(parameters, mu_eq, num_mu, qs_eq, num_qs):
    # Solve the model using the optimal parameters
    time_pred, biomass_pred, substrate_pred, volume_pred = model_optimization(param, parameters, mu_eq, num_mu, qs_eq, num_qs)  # Solve the model using the current parameters
    biomass = pd.concat([biomass_exp, pd.Series(biomass_pred)], axis=1, keys=['biomass_exp', 'biomass_pred']).dropna()
    biomass_exp_ = biomass['biomass_exp'].values
    biomass_pred_ = biomass['biomass_pred'].values
    mse_x = mean_squared_error(biomass_exp_, biomass_pred_)  # Calculate mean squared error for biomass

    glucose = pd.concat([substrate_exp, pd.Series(substrate_pred)], axis=1, keys=['substrate_exp', 'substrate_pred']).dropna()
    substrate_exp_ = glucose['substrate_exp'].values
    substrate_pred_ = glucose['substrate_pred'].values
    mse_s = mean_squared_error(substrate_exp_, substrate_pred_)  # Calculate mean squared error for substrate
    
    # Calculate the combined rmse
    mse = (mse_x + mse_s)/2
    rmse = np.sqrt(mse)  # Calculate root mean squared error
    return rmse, time_pred, biomass_pred, substrate_pred, volume_pred

##### 4. Define set of parameters in config file

In [25]:
# Extract the parameter combination and the number of the set in order to save it in the correct folder
est_mu_max = param['est_mu_max'] # ['set0']
est_Ks = param['est_ks']

In [26]:
# Set the number of samples and parameters
num_samples = 100
num_parameters = 9

# Define the ranges for each parameter
parameter_bounds = [
    [0.01, 0.6],    # Range for parameter 1 mu_max
    [18, 25],       # Range for parameter 2 X_max
    [0.1, 10.0],    # Range for parameter 3 - Ks
    [0.01, 0.1],    # Range for parameter 4 - Ks_qs
    [6.0, 8.0],     # Range for parameter 5 - Ki
    [0.4, 0.5],     # Range for parameter 6 - Yxs
    [0.5, 1.5],     # Range for parameter 7 - qs_max
    [0.05, 0.15],   # Range for parameter 8 - m_s
    [0.001, 0.02],    # Range for parameter 9 - lag
]

In [27]:
LHS_samples = get_LHS_samples(num_samples, num_parameters, parameter_bounds)
LHS_samples.shape

(100, 9)

##### 5. Run it

In [29]:
df_all_sets = pd.DataFrame(columns=['set', 'mu', 'qs', 'mu_max', 'X_max', 'Ks', 'Ks_qs', 'Ki', 'Yxs', 'qs_max', 'm_s', 'lag', 'rmse'])
for set_num in range(LHS_samples.shape[0]):
    # Save all parameters and equations and the RMSE in a dataframe
    ## with the beginning of one set a new rmse_overview will be created
    rmse_one_set = []
    #key = f'set{set_num}' ; init_p = est_mu_max[key]
    init_p = list(LHS_samples[set_num, :])
    for i in range(len(mu_all)):
        for j in range(len(qs_all)):
            # Define the combination of equations
            mu_eq = mu_all[i]; num_mu = i
            qs_eq = qs_all[j]; num_qs = j

            # Make the predictions and calculate the error
            rmse, time_pred, biomass_pred, substrate_pred, volume_pred = objective_function(init_p, mu_eq, num_mu, qs_eq, num_qs)
            # save the parameters in a dataframe
            append_list=[set_num, i, j, init_p[0], init_p[1], init_p[2], init_p[3], init_p[4], init_p[5], init_p[6], init_p[7], init_p[8], round(rmse, 3)]
            rmse_one_set.append(append_list)

            # Make a plot and save it
            title = f'set{set_num}/ mu{i} - qs{j} - rmse: {round(rmse, 3)}'
            plot_name = f'mu{i}_qs{j}'

            if rmse <= float(20):
                plot_estimation(time_pred, biomass_pred, substrate_pred, volume_pred, title, plot_name, set_num)

    # save the parameters of one set
    df_1set = pd.DataFrame(rmse_one_set, columns=['set', 'mu', 'qs', 'mu_max', 'X_max', 'Ks', 'Ks_qs', 'Ki', 'Yxs', 'qs_max', 'm_s', 'lag', 'rmse'])
    df_all_sets = pd.concat([df_all_sets,df_1set], axis=0, ignore_index=True)


df_all_sets.sort_values(by=['rmse'], ascending=True, inplace=True)
df_all_sets.to_csv(f'data/estimation/LHS_sampling/data.csv')

df_all_sets.head(10)

[0.1485238478336399, 22.293301608131287, 9.870131827970152, 0.08120136377981549, 7.010735086121652, 0.46505263310132033, 1.2707492630192125, 0.14290283536057755, 0.012808668246171703]


C:\Users\dxp2904\AppData\Local\Temp\ipykernel_13944\465046859.py:8: RuntimeWarning: overflow encountered in exp
  qs3 = lambda qs_max, c_glucose, Ks_qs, glu_met, lag: qs_max * c_glucose / (Ks_qs + c_glucose) * (1 / (np.exp(glu_met * lag)))


[0.3203148688136113, 19.58582572354488, 0.4832295681056016, 0.022120926625127227, 6.329364487978653, 0.41126209558748084, 0.8050920894514602, 0.14495247685821577, 0.011153158556868]
[0.542857164617334, 20.25897667024924, 3.352407629259761, 0.08091578962980754, 6.985637429253727, 0.40456470902946845, 0.8974786888695617, 0.0655756214704503, 0.008454560907614352]
[0.4409398042063017, 18.364935282136916, 1.6153110571787128, 0.054500766559563714, 6.712595022727813, 0.4754819237308886, 1.085994966811608, 0.10897190293925668, 0.005349419962728184]


C:\Users\dxp2904\AppData\Local\Temp\ipykernel_13944\465046859.py:8: RuntimeWarning: overflow encountered in exp
  qs3 = lambda qs_max, c_glucose, Ks_qs, glu_met, lag: qs_max * c_glucose / (Ks_qs + c_glucose) * (1 / (np.exp(glu_met * lag)))


[0.397751227589453, 23.067518192861684, 2.4520581474194585, 0.01618395116586707, 6.780164662471706, 0.4831594026649274, 0.7913392644382143, 0.06388572247416505, 0.014114872661511603]


...

C:\Users\dxp2904\AppData\Local\Temp\ipykernel_13944\465046859.py:8: RuntimeWarning: overflow encountered in exp
  qs3 = lambda qs_max, c_glucose, Ks_qs, glu_met, lag: qs_max * c_glucose / (Ks_qs + c_glucose) * (1 / (np.exp(glu_met * lag)))


[0.16989574867966095, 24.034723397273154, 6.005799355936087, 0.035372897998450746, 7.4967513272054465, 0.4068822576852017, 1.1567906314649763, 0.07337865620353277, 0.003583581063866901]


C:\Users\dxp2904\AppData\Local\Temp\ipykernel_13944\465046859.py:8: RuntimeWarning: overflow encountered in exp
  qs3 = lambda qs_max, c_glucose, Ks_qs, glu_met, lag: qs_max * c_glucose / (Ks_qs + c_glucose) * (1 / (np.exp(glu_met * lag)))


[0.05201690212901669, 19.858246909595596, 7.734488106244522, 0.08443863993379498, 6.622358799247081, 0.4370019547417434, 0.6325550202335586, 0.14888640883568832, 0.010119849670988234]


,set,mu,qs,mu_max,X_max,Ks,Ks_qs,Ki,Yxs,qs_max,m_s,lag,rmse
951,59,1,3,0.151842,21.610650,1.849366,0.077962,7.402749,0.499585,1.136157,0.121758,0.017414,3.832
871,54,1,3,0.136699,18.216403,0.535589,0.076009,6.491255,0.450012,1.430207,0.110820,0.016074,3.834
1540,96,1,0,0.181003,19.182082,6.158906,0.020521,7.455448,0.457738,0.756964,0.115150,0.010964,3.907
7,0,1,3,0.148524,22.293302,9.870132,0.081201,7.010735,0.465053,1.270749,0.142903,0.012809,4.050
727,45,1,3,0.218356,18.659077,1.210381,0.055047,6.178527,0.454024,0.723849,0.076236,0.012178,4.171
772,48,1,0,0.139866,23.463812,6.253094,0.027281,7.919310,0.420863,1.160831,0.120088,0.012483,4.182
1100,68,3,0,0.367979,24.948018,0.221270,0.099580,6.306802,0.402236,1.236294,0.096955,0.016771,4.395
424,26,2,0,0.286853,19.421220,0.714451,0.095325,6.108962,0.490747,0.676160,0.054178,0.018813,4.737
1468,91,3,0,0.459201,22.395016,1.482734,0.082202,7.032032,0.464889,1.413100,0.140962,0.014582,5.084
775,48,1,3,0.139866,23.463812,6.253094,0.027281,7.919310,0.420863,1.160831,0.120088,0.012483,5.184


In [30]:
df_LHS = pd.read_csv('data/estimation/LHS_sampling/data.csv')